In [17]:
!pip install boto3 requests
!import numpy as np
!pip install transformers datasets faiss-cpu boto3 requests
!pip install langchain
!pip install -U langchain-community
!pip install langchain_together
!pip install sentence-transformers
!pip install python-docx
!pip install pandas python-docx
!!pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


import: unable to open X server `' @ error/import.c/ImportImageCommand/348.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


['Requirement already satisfied: faiss-cpu in /opt/conda/lib/python3.10/site-packages (1.9.0.post1)',
 'Requirement already satisfied: numpy<3.0,>=1.25.0 in /opt/conda/lib/python3.10/site-packages (from faiss-cpu) (1.26.4)',
 'Requirement already satisfied: packaging in /opt/conda/lib/python3.10/site-packages (from faiss-cpu) (24.2)']

In [18]:
#implemented chunking below

In [19]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
import time
tic=time.time()

In [21]:
import os
from docx import Document as DocxDocument  # Rename to avoid conflict
import numpy as np
import faiss
from langchain.chains import LLMChain
from langchain_together import ChatTogether
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings  # Correct import from langchain
from langchain.docstore import InMemoryDocstore
from langchain.docstore.document import Document  # langchain's Document for docstore

# Step 1: Read the Word file
word_file_path = '/kaggle/input/newdata/FDKP.docx'
doc = DocxDocument(word_file_path)  # Use renamed class

# Step 2: Extract text and chunk the document (using paragraphs as chunks)
full_text = []
for para in doc.paragraphs:
    full_text.append(para.text)

# Split the document into chunks (e.g., paragraphs in this case)
chunks = full_text  # Each chunk is a paragraph

# Step 3: Initialize the LLM model (using your API key and model)
chat_model = ChatTogether(
    together_api_key="c51c9bcaa6bf7fae3ce684206311564828c13fa2e91553f915fee01d517ccee9",
    model="meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo",
)

# Step 4: Define your prompt template
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "You are an expert financial advisor. Use the context to answer questions accurately and concisely.\n"
        "Context: {context}\n\n"
        "Question: {question}\n\n"
        "Answer (be specific and avoid hallucinations):"
    )
)

# Step 5: Load the embedding model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")  # Correct import from langchain

# Step 6: Create embeddings for the chunks
embeddings = embedding_model.embed_documents(chunks)
embeddings = np.array(embeddings)
dimension = embeddings.shape[1]  # Embedding dimension

# Step 7: Initialize FAISS index and add embeddings
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Step 8: Setup the docstore and mapping between index and document chunks
# Fixed the `Document` instantiation to use `page_content` instead of `content`
docstore = InMemoryDocstore({i: Document(page_content=chunks[i]) for i in range(len(chunks))})
index_to_docstore_id = {i: i for i in range(len(chunks))}

# Step 9: Initialize the FAISS vector store
vector_store = FAISS(
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id,
    embedding_function=embedding_model.embed_query
)

In [22]:
# Define the QA chain for answering questions
qa_chain = LLMChain(
    llm=chat_model,
    prompt=prompt_template
)

# Example usage
if __name__ == "__main__":
    # Define the question
    question = "What is SDP"#"What is the interest rate for senior citizens for FD?"

    # Create an embedding for the query
    query_embedding = embedding_model.embed_query(question)
    
    # Search the vector store for the most similar documents
    D, I = index.search(np.array([query_embedding]), k=1)
    
    # Retrieve the most relevant document
    doc_id = I[0][0]
    document = docstore.search(doc_id)
    context = document.page_content
    
    # Use the qa_chain to get an answer
    answer = qa_chain.run(context=context, question=question, clean_up_tokenization_spaces=False)

    # Print the result
    print(answer)


The Systematic Deposit Plan (SDP). It's a plan where customers can invest a fixed amount, as low as Rs. 5,000, every month for a specified period of time (19 to 60 months).


In [23]:
print((time.time()-tic)/60)

0.23814497788747152


In [24]:
def fun1(question):
    # Define the question
    question = question#"What is SDP"#"What is the interest rate for senior citizens for FD?"

    # Create an embedding for the query
    query_embedding = embedding_model.embed_query(question)
    
    # Search the vector store for the most similar documents
    D, I = index.search(np.array([query_embedding]), k=1)
    
    # Retrieve the most relevant document
    doc_id = I[0][0]
    document = docstore.search(doc_id)
    context = document.page_content
    
    # Use the qa_chain to get an answer
    answer = qa_chain.run(context=context, question=question, clean_up_tokenization_spaces=False)

    # Print the result
    return(answer,context)


In [25]:
fun1("For what tenure I can get maximum FD rates in Bajaj Finance?")

('You can get the maximum FD rates in Bajaj Finance for a tenure of 42 months.',
 'With Bajaj Finance you can invest for a tenure of 12-60 months. The highest FD rates are offered for a tenure of 42 months, which is up to 8.65% p.a. for senior citizens and up to 8.40% p.a. for customers below 60 years of age.')

In [26]:
###implemented top n below

In [27]:
import os
from docx import Document as DocxDocument  # Rename to avoid conflict
import numpy as np
import faiss
from langchain.chains import LLMChain
from langchain_together import ChatTogether
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings  # Correct import from langchain
from langchain.docstore import InMemoryDocstore
from langchain.docstore.document import Document  # langchain's Document for docstore
import time 
tic=time.time()

# Step 1: Read the Word file
word_file_path = '/kaggle/input/newdata/FDKP.docx'
doc = DocxDocument(word_file_path)  # Use renamed class

# Step 2: Extract text and chunk the document (using paragraphs as chunks)
full_text = []
for para in doc.paragraphs:
    full_text.append(para.text)

# Split the document into chunks (e.g., paragraphs in this case)
chunks = full_text  # Each chunk is a paragraph

# Step 3: Initialize the LLM model (using your API key and model)
chat_model = ChatTogether(
    together_api_key="c51c9bcaa6bf7fae3ce684206311564828c13fa2e91553f915fee01d517ccee9",
    model="meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo",
)

# Step 4: Define your prompt template
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "You are an expert financial advisor. Use the context to answer questions accurately and concisely.\n"
        "Context: {context}\n\n"
        "Question: {question}\n\n"
        "Answer (be specific and avoid hallucinations):"
    )
)

# Step 5: Load the embedding model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")  # Correct import from langchain

# Step 6: Create embeddings for the chunks
embeddings = embedding_model.embed_documents(chunks)
embeddings = np.array(embeddings)
dimension = embeddings.shape[1]  # Embedding dimension

# Step 7: Initialize FAISS index and add embeddings
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Step 8: Setup the docstore and mapping between index and document chunks
# Fixed the `Document` instantiation to use `page_content` instead of `content`
docstore = InMemoryDocstore({i: Document(page_content=chunks[i]) for i in range(len(chunks))})
index_to_docstore_id = {i: i for i in range(len(chunks))}

# Step 9: Initialize the FAISS vector store
vector_store = FAISS(
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id,
    embedding_function=embedding_model.embed_query
)

# Define the QA chain for answering questions
qa_chain = LLMChain(
    llm=chat_model,
    prompt=prompt_template
)

# Example usage
if __name__ == "__main__":
    # Define the question
    question = "What is SDP"  # Example question

    # Define the number of top documents you want to retrieve
    top_n = 5  # For example, retrieve the top 5 most similar documents

    # Step 10: Create an embedding for the query
    query_embedding = embedding_model.embed_query(question)

    # Step 11: Search the vector store for the top_n most similar documents
    D, I = index.search(np.array([query_embedding]), k=top_n)

    # Step 12: Retrieve the most relevant documents from docstore
    relevant_docs = []
    for doc_id in I[0]:
        document = docstore.search(doc_id)
        relevant_docs.append(document.page_content)

    # Step 13: Combine all relevant documents as context (optional)
    context = "\n".join(relevant_docs)

    # Step 14: Use the qa_chain to get an answer
    answer = qa_chain.run(context=context, question=question, clean_up_tokenization_spaces=False)

    # Step 15: Print the result
    print(answer)

Systematic Deposit Plan (SDP) is a plan that allows you to invest a fixed sum regularly over a specific tenure.


In [28]:
context # top_n=5

'Systematic Deposit Plan - Single Maturity Scheme (SMS) We have created a special deposit plan for customers interested in making regular deposits, referred to as a Systematic Deposit Plan (SDP). In an SDP, you can invest as low as Rs. 5,000 every month for a defined tenure (19 to 60 months). Under the Single Maturity Scheme (SMS), you get the \'principal + interest\' at maturity. The interest on each new deposit is calculated based on the prevailing interest rates in the deposit month.\nSystematic Deposit Plan - Monthly Maturity Scheme (MMS) We have created a special deposit plan for customers interested in making recurring deposits, referred to as a Systematic Deposit Plan (SDP). In an SDP, you can invest as low as Rs. 5,000 every month for a defined tenure (12 to 60 months). Under the Monthly Maturity Scheme (MMS), \'principal + interest\' for each deposit is paid monthly, on maturity of each deposit.\nYou can deposit a fixed sum for a specific period with Bajaj Finance Fixed Deposi

In [30]:
print((time.time()-tic)/60)

0.23833266496658326


In [34]:
def qry(qry1):    # Define the question
    question = qry1#"What is SDP"  # Example question

    # Define the number of top documents you want to retrieve
    top_n = 5  # For example, retrieve the top 5 most similar documents

    # Step 10: Create an embedding for the query
    query_embedding = embedding_model.embed_query(question)

    # Step 11: Search the vector store for the top_n most similar documents
    D, I = index.search(np.array([query_embedding]), k=top_n)

    # Step 12: Retrieve the most relevant documents from docstore
    relevant_docs = []
    for doc_id in I[0]:
        document = docstore.search(doc_id)
        relevant_docs.append(document.page_content)

    # Step 13: Combine all relevant documents as context (optional)
    context = "\n".join(relevant_docs)

    # Step 14: Use the qa_chain to get an answer
    answer = qa_chain.run(context=context, question=question, clean_up_tokenization_spaces=False)

    # Step 15: Print the result
    print(answer)

In [36]:
qry("For what tenure is the max interest rate in Bajaj FD?")

The maximum interest rate of 8.65% p.a. for senior citizens is offered on a tenure of 42 months.


In [42]:
qry("What are senior citizen FD rates at Bajaj Finance and for what tenure?")

The senior citizen FD rates at Bajaj Finance are up to 8.65% p.a. for a special tenure of 42 months.


In [45]:
qry("42 months FD interest rate for senior citizen at Bajaj Finance please?")

The interest rate for a 42-month FD for senior citizens at Bajaj Finance is 8.65% at maturity, 8.33% monthly, 8.38% quarterly, 8.47% half-yearly, and 8.65% annually.


In [32]:
from nltk.translate.bleu_score import sentence_bleu

# Sample outputs
base_model = ["Systematic Deposit Plan (SDP) is a deposit plan that allows you to invest a fixed sum regularly over a specific tenure"]
model_rag_chunk = ["Systematic Deposit Plan (SDP), also referred to as Bajaj Finance Fixed Deposit, is a plan that allows you to invest a fixed sum regularly over a specific tenure, and you can earn interest upon maturity or regular payouts."]
model_rag_graph = ["[c: 7.50%, d: Senior Citizen Deposit Plan (SDP) offers a higher interest rate of 7.50% for senior citizens.]\n\nMy reasoning is that the question asks about SDP, and the context provides a specific interest rate and description for the Senior Citizen Deposit Plan, which matches the query."]

# Function to calculate BLEU score
def calculate_bleu(reference, candidate):
    reference = [reference[0].split()]  # Convert reference to a list of words
    candidate = candidate[0].split()     # Convert candidate to a list of words
    score = sentence_bleu(reference, candidate)
    return score

# Calculate BLEU scores
bleu_llm1 = calculate_bleu(base_model, model_rag_chunk)
bleu_llm2 = calculate_bleu(base_model, model_rag_graph)

print(f"BLEU score for LLM1: {bleu_llm1}")
print(f"BLEU score for LLM2: {bleu_llm2}")

BLEU score for LLM1: 0.36774543142109295
BLEU score for LLM2: 0.12532443621763042


In [37]:
base_model = ["The maximum FD rate in Bajaj Finance is offered for a tenure of 42 months"]
model_rag_chunk = ["The maximum interest rate of 8.65% p.a. for senior citizens is offered on a tenure of 42 months."]
model_rag_graph = ["The maximum interest rate for Bajaj Finance's fixed deposit is offered for a tenure of 5 years.\n\nReasoning:\n- The context provides interest rates for different tenures, and the highest interest rate is mentioned for the 5-year tenure.\n- This suggests that the 5-year tenure is the optimal choice for maximizing returns on investment in Bajaj Finance's fixed deposit."]

bleu_llm1 = calculate_bleu(base_model, model_rag_chunk)
bleu_llm2 = calculate_bleu(base_model, model_rag_graph)

print(f"BLEU score for LLM1: {bleu_llm1}")
print(f"BLEU score for LLM2: {bleu_llm2}")

BLEU score for LLM1: 0.1920953415125867
BLEU score for LLM2: 0.09590301722906841


In [43]:
base_model = ["Bajaj Finance offers senior citizen FD rates up to 8.65% p.a., specifically for a tenure of 42 months."]
model_rag_chunk = ["The senior citizen FD rates at Bajaj Finance are up to 8.65% p.a. for a special tenure of 42 months."]
model_rag_graph = ["Senior Citizen FD rates at Bajaj Finance are as follows:\n- 7.50% for 36 months\n- 7.75% for 60 months\n- 8.00% for 84 months\n\nThese rates are relevant to the question because they specifically mention senior citizen FD rates, which is what the user is inquiring about. The tenures mentioned (36 months, 60 months, and 84 months) are also relevant as they provide the user with information on the available options for different investment periods"]

bleu_llm1 = calculate_bleu(base_model, model_rag_chunk)
bleu_llm2 = calculate_bleu(base_model, model_rag_graph)

print(f"BLEU score for LLM1: {bleu_llm1}")
print(f"BLEU score for LLM2: {bleu_llm2}")

BLEU score for LLM1: 0.3370129264673147
BLEU score for LLM2: 0.09611672870481912


In [46]:
base_model = ["8.65% at maturity, 8.33% monthly, 8.38% quarterly, 8.47% half-yearly, and 8.65% annually"]
model_rag_chunk = ["The interest rate for a 42-month FD for senior citizens at Bajaj Finance is 8.65% at maturity, 8.33% monthly, 8.38% quarterly, 8.47% half-yearly, and 8.65% annually"]
model_rag_graph = ["5 years: [c: 7.50%, d: For Senior Citizens]\n* 3 years: [c: 7.25%, d: For Senior Citizens]\n* 5 years: [c: 7.50%, d: For General Public]\n* 3 years: [c: 7.25%, d: For General Public]\n\nMy reasoning is that the question asks for the interest rate for a 42-month FD specifically for senior citizens at Bajaj Finance. However, the exact duration of 42 months is not mentioned in the context. The closest durations mentioned are 3 years and 5 years. Since the question is specifically for senior citizens, I have selected the interest rates for 3 years and 5 years for senior citizens"]

bleu_llm1 = calculate_bleu(base_model, model_rag_chunk)
bleu_llm2 = calculate_bleu(base_model, model_rag_graph)

print(f"BLEU score for LLM1: {bleu_llm1}")
print(f"BLEU score for LLM2: {bleu_llm2}")

BLEU score for LLM1: 0.42657068031281004
BLEU score for LLM2: 0.37329157887410686
